Data collection
Substracting the list of fifa ranking from FIFA official site.

In [4]:
pip install beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install aiohttp

Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install requests

Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install requests-html

Note: you may need to restart the kernel to use updated packages.


postgreSQL 데이터 베이스
code ex)
cur.execute("INSERT INTO test_table (name, age)VALUES(%s, %s),(name,age))

In [8]:
pip install psycopg2

Note: you may need to restart the kernel to use updated packages.


In [9]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [10]:
import datetime
import asyncio
import aiohttp
import requests as r
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup, SoupStrainer
import nest_asyncio
nest_asyncio.apply()

In [11]:
from requests_html import HTMLSession

In [12]:
fifa_url = 'https://www.fifa.com/fifa-world-ranking/men?dateId=id13554'
session = HTMLSession()
page = session.get(fifa_url)
soup = BeautifulSoup(page.content, 'html.parser')
rk_elements = soup.find('tbody',class_='fc-ranking-item-full_rankingTableFullRow__1nbp7')

def scrapy_rank_table(page):
    rows = BeautifulSoup(page, 
                          'html.parser', 
                          parse_only=SoupStrainer('tbody')).find_all('tr')
    table = []
    for row in rows:
        table.append({
            'id': int(row['data-team-id']), 
            'country_full': row.find('span', {'class': 'fi-t__nText'}).text, 
            'country_abrv': row.find('span', {'class': 'fi-t__nTri'}).text,
            'rank': int(row.find('td', {'class': 'fi-table__rank'}).text), 
            #'total_points': int(row.find('td', {'class': 'fi-table__points'}).text),
            'total_points': row.find('td', {'class': 'fi-table__points'}).text,
            #'previous_points': int(row.find('td', {'class': 'fi-table__prevpoints'}).text or 0),
            'previous_points': row.find('td', {'class': 'fi-table__prevpoints'}).text or 0,
            'rank_change': int(row.find('td', {'class': 'fi-table__rankingmovement'}).text.replace('-', '0')),
            'confederation': row.find('td', {'class': 'fi-table__confederation'}).text.strip('#')
        })
    return table

CSV 파일 데이터 베이스 옮기기

In [13]:
import psycopg2
import csv

conn = psycopg2.connect(
    host="kashin.db.elephantsql.com",
    database="wqtyydnx",
    user="wqtyydnx",
    password="cyK_N1rT0ijTZRC3EtzPMaOTFME6Gaob")
conn.autocommit = True
cur = conn.cursor()

In [14]:
cur.execute('''DROP TABLE IF EXISTS fifa_ranking_21;''')
fifa_ranking = """CREATE TABLE fifa_ranking_21(
    id int,
    rank INT,
    country_full VARCHAR,
    country_abrv VARCHAR,
    total_points INT,
    previous_points  INT,
    rank_change INT,
    confederation VARCHAR,
    rank_date DATE);"""

cur.execute(fifa_ranking)



In [15]:
cur.execute('''DROP TABLE IF EXISTS results;''')
results = """CREATE TABLE results(
    date DATE,
    home_team VARCHAR,
    away_team VARCHAR,
    home_score INT,
    away_score INT,
    tournament VARCHAR,
    city VARCHAR,
    country VARCHAR,
    neutral VARCHAR);"""

cur.execute(results)

### API 개발 ### 
### 2022 카타르 월드컵 승리 예측 모델 ###

예상 진출 국가

In [16]:
#AFC
#CAF
#CONCACAF
#CONMEBOL
#OFC
#UEFA
teams = [['Qatar', 'QAT', 'AFC'],
        ['Iran', 'IRN', 'AFC'],
        ['South Korea', 'KOR', 'AFC'],
        ['United Arab Emirates', 'UAE', 'AFC'],
        ['Saudi Arabia', 'KSA', 'AFC'],
        ['Japan', 'JPN', 'AFC'],
        ['Australia', '	AUS', 'AFC'],
        ['Canada', 'CAN', 'CAF'],
        ['United States', '	USA', 'CAF'],
        ['Mexico', 'MEX', 'CAF'],
        ['Panama', 'PAN', 'CAF'],
        ['Brazil', 'BRA', 'CONMEBOL'],
        ['Argentina', 'ARG', 'CONMEBOL'],
        ['Ecuador', 'ECU', 'CONMEBOL'],
        ['Uruguay', ' URU', 'CONMEBOL'],
        ['Peru', 'PER', 'CONMEBOL'],
        ['Solomon Islands', 'SOL', 'OFC'],
        ['Tahiti', 'TAH', 'OFC'],
        ['New Zealand', 'NZL', 'OFC'],
        ['New Caledonia', 'NCL', 'OFC'],
        ['Serbia', 'SRB', 'UEFA'],
        ['Portugal', 'POR', 'UEFA'],
        ['Spain', 'ESP', 'UEFA'],
        ['Sweden', 'SWE', 'UEFA'],
        ['Switzerland', 'SUI', 'UEFA'],
        ['Italy', 'ITA', 'UEFA'],
        ['France', 'FRA', 'UEFA'],
        ['Ukraine', 'UKR', 'UEFA'],
        ['Belgium', 'BEL', 'UEFA'],
        ['Wales', 'WAL', 'UEFA'],
        ['Czech Republic', 'CZE', 'UEFA'],
        ['Denmark', 'DEN', 'UEFA'],
        ['Scotland', 'SCO', 'UEFA'],
        ['Austria', 'AUT', 'UEFA'],
        ['Netherlands', 'NED', 'UEFA'],
        ['Turkey', 'TUR', 'UEFA'],
        ['Croatia', 'CRO', 'UEFA'],
        ['Russia', 'RUS', 'UEFA'],
        ['England', 'ENG', 'UEFA'],
        ['Poland', 'POL', 'UEFA'],
        ['Germany', 'GER', 'UEFA'],
        ['North Macedonia', 'MKD', 'UEFA']]

teams_df = pd.DataFrame(teams, columns=['Country', 'Abrv', 'Confederation'])
teams_df.head(4)

Country Abrv Confederation
0                 Qatar  QAT           AFC
1                  Iran  IRN           AFC
2           South Korea  KOR           AFC
3  United Arab Emirates  UAE           AFC

In [17]:
match_df = pd.read_csv('/Users/Hyungdanny/Desktop/ai/project03/csv/results.csv')
# only matches from 1993
match_df = match_df[match_df.date > '1993-01-01']
# change country name
match_df['home_team'] = match_df['home_team'].replace({'Czechoslovakia' : 'Czech Republic', 'German DR': 'Germany'})
match_df['away_team'] = match_df['away_team'].replace({'Czechoslovakia' : 'Czech Republic', 'German DR': 'Germany'})
# only matches between playing teams
match_df = match_df[match_df.home_team.isin(teams_df.Country.to_list())]
match_df = match_df[match_df.away_team.isin(teams_df.Country.to_list())]
match_df.reset_index(drop=True, inplace=True)
# drop city column
match_df.drop(labels='city', axis=1, inplace=True)
print('Number of matches after filtering: {}'.format(len(match_df)))

Number of matches after filtering: 3196


In [18]:
match_df.head(10)

date      home_team    away_team  home_score  away_score  \
0  1993-01-20          Italy       Mexico           2           0   
1  1993-01-27          Spain       Mexico           1           1   
2  1993-01-30  United States      Denmark           2           2   
3  1993-02-13  United States       Russia           0           1   
4  1993-02-18      Argentina       Brazil           1           1   
5  1993-02-21  United States       Russia           0           0   
6  1993-02-24      Argentina      Denmark           1           1   
7  1993-02-24    Netherlands       Turkey           3           1   
8  1993-02-24       Portugal        Italy           1           3   
9  1993-03-09         Canada  South Korea           0           2   

                     tournament        country  neutral  
0                      Friendly          Italy    False  
1                      Friendly          Spain    False  
2                      Friendly  United States    False  
3                      Friendly  United States    False  
4                      Friendly      Argentina    False  
5                      Friendly  United States    False  
6                      Friendly      Argentina    False  
7  FIFA World Cup qualification    Netherlands    False  
8  FIFA World Cup qualification       Portugal    False  
9                      Friendly         Canada    False

In [19]:
#fifa ranking df
fifa_ranking_df = pd.read_csv('/Users/Hyungdanny/Desktop/ai/project03/csv/fifa_ranking-2021-05-27.csv')
fifa_ranking_df.head()

id  rank         country_full country_abrv  total_points  \
0    43818    82                 Iraq          IRQ            15   
1    43873   107           Mozambique          MOZ             9   
2    43816   108            Indonesia          IDN             9   
3  1882218   109  Antigua and Barbuda          ATG             8   
4    43820   110               Jordan          JOR             8   

   previous_points  rank_change confederation   rank_date  
0                0            0           AFC  1992-12-31  
1                0            0           CAF  1992-12-31  
2                0            0           AFC  1992-12-31  
3                0            0      CONCACAF  1992-12-31  
4                0            0           AFC  1992-12-31

In [20]:
def data_correction(df):
    df.replace({'country_full': 'Korea Republic'}, 'South Korea', inplace=True)
    df.replace({'country_full': 'Czechoslovakia'}, 'Czech Republic', inplace=True)
    return df

In [21]:
fifa_ranking_df = data_correction(fifa_ranking_df)

In [22]:
MATCHES = len(match_df)

home_rank = np.zeros(MATCHES, dtype=np.int)
away_rank = np.zeros(MATCHES, dtype=np.int)
home_total_points = np.zeros(MATCHES, dtype=np.float)
away_total_points = np.zeros(MATCHES, dtype=np.float)
for i in range(MATCHES):
    home_listing = fifa_ranking_df[((fifa_ranking_df.country_full == match_df.iloc[i].home_team) & 
                            (fifa_ranking_df.rank_date <= match_df.iloc[i].date))].sort_values(by='rank_date', ascending=False)
    
    try:
        home_rank[i] = int(home_listing.iloc[0]['rank'])
        home_total_points[i] = int(home_listing.iloc[0]['total_points'])
    except:
        home_rank[i] = 155
        
    away_listing = fifa_ranking_df[((fifa_ranking_df.country_full == match_df.iloc[i].away_team) & 
                            (fifa_ranking_df.rank_date <= match_df.iloc[i].date))].sort_values(by='rank_date', ascending=False)
        
    try:
        away_rank[i] = int(away_listing.iloc[0]['rank'])
        away_total_points[i] = int(home_listing.iloc[0]['total_points'])
    except:
        away_rank[i] = 155

/var/folders/hn/b0fcr4051tq4wdm6f67dndsr0000gn/T/ipykernel_11293/4138010732.py:3: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  home_rank = np.zeros(MATCHES, dtype=np.int)
/var/folders/hn/b0fcr4051tq4wdm6f67dndsr0000gn/T/ipykernel_11293/4138010732.py:4: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review yo

In [23]:
match_df['home_rank'] = home_rank
match_df['away_rank'] = away_rank
match_df['friendly'] = (match_df.tournament == 'Friendly')
match_df['FIFA World Cup'] = (match_df.tournament == 'FIFA World Cup')
match_df['home_point'] = home_total_points
match_df['away_point'] = away_total_points
match_df.drop(labels=['tournament', 'date', 'country'], axis=1, inplace=True)
match_df.neutral = match_df.neutral.astype(int)
match_df.friendly = match_df.friendly.astype(int)
match_df['FIFA World Cup'] = match_df['FIFA World Cup'].astype(int)


XGboost 모델

In [24]:
from xgboost import XGBRegressor
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder
from category_encoders import OrdinalEncoder
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.preprocessing import StandardScaler

/Users/Hyungdanny/opt/anaconda3/envs/03_01/lib/python3.10/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [25]:
features = ['home_team','away_team','neutral',
            'home_rank','away_rank','friendly',
            'FIFA World Cup','home_point','away_point']
target01 = ['home_score']
target02 = ['away_score']

X = match_df[features]
h_y = match_df[target01]
a_y = match_df[target02]

In [26]:
onehot_columns = ['home_team', 'away_team']
onehot_df = X[onehot_columns]
onehot_df = pd.get_dummies(onehot_df, columns = onehot_columns)
match_onehot_drop = X.drop(onehot_columns, axis = 1)
match_onehot = pd.concat([match_onehot_drop, onehot_df], axis = 1)

In [27]:
from xgboost import XGBRegressor as xgb

# home team score model
h_model = xgb()
h_model.fit(match_onehot.values, h_y.values)
#away team score model
a_model = xgb()
a_model.fit(match_onehot.values, a_y.values)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.300000012,
             max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=100, n_jobs=12,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)

In [28]:
def predict(h_country, a_country, neutral=True):
    # create vector
    cols = ['neutral', 'home_rank', 'away_rank', 'friendly','FIFA World Cup','home_point','away_point']
    for c in onehot_df.columns.to_list():
        cols.append(c)
    df = pd.DataFrame(np.zeros((1,len(cols)), dtype=int), columns=cols)
    if neutral:
        df.neutral.iloc[0] = 1
    else:
        df.neutral.iloc[0] = 0
    df.home_rank.iloc[0] = fifa_ranking_df[((fifa_ranking_df.rank_date == '2021-05-27') & (fifa_ranking_df.country_full == h_country))]['rank'].values[0]
    df.away_rank.iloc[0] = fifa_ranking_df[((fifa_ranking_df.rank_date == '2021-05-27') & (fifa_ranking_df.country_full == a_country))]['rank'].values[0]
    df['home_team_'+h_country].iloc[0] = 1
    df['away_team_'+a_country].iloc[0] = 1
    #df = df[hmodel.get_booster().feature_names]
    # predict
    hscore = int(h_model.predict(df.iloc[0].to_numpy().reshape(1,91))[0])
    ascore = int(a_model.predict(df.iloc[0].to_numpy().reshape(1,91))[0])
    return hscore, ascore



승부 예측

In [29]:
predict('South Korea','France',True)

(0, 4)

In [35]:
import pickle

with open('h_model.pkl','wb') as h_pk:
    pickle.dump(h_model,h_pk)

In [36]:
with open('a_model.pkl','wb') as a_pk:
    pickle.dump(a_model,a_pk)

In [37]:
with open('onehot_df.pkl','wb') as onehot:
    pickle.dump(onehot_df,onehot)

In [38]:
with open('fifa_ranking_df.pkl','wb') as fifa_ranking_df:
    pickle.dump(fifa_ranking_df,fifa_ranking_df)

TypeError: cannot pickle '_io.BufferedWriter' object